In [8]:
%%capture
%pip install sentence_transformers
%pip install dataset

In [2]:
%%capture
#Environment: FAIR-OER(python3.8.18)
#LIBRARIES
import random
import pandas as pd
import ast
import pytz
import re
from datetime import datetime
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import mean_squared_error, r2_score
#import pdftotext
#from googletrans import Translator
#from langdetect import detect, LangDetectException
import numpy as np
import torch
#from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine
from sklearn.metrics import roc_curve, auc, roc_auc_score
#import seaborn as sns
import scipy.stats as stats
from scipy.stats import skew
from scipy.stats import kurtosis
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pyarrow
import pyarrow.parquet
#from datasets import load_dataset
#from datasets import Features, Value
#from sentence_transformers import SentenceTransformer, losses
#from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
#from sentence_transformers.similarity_functions import SimilarityFunction
#from sentence_transformers.trainer import SentenceTransformerTrainer
#from sentence_transformers.training_args import SentenceTransformerTrainingArguments

In [3]:
survey = pd.read_excel(R"G:\Other computers\My Laptop\B\Publication\FAIR OER\02-Data\02-Analyzed\surveys\Experts-Results.xlsx",sheet_name='all_V1.2')

In [4]:
survey

,pair 1-tum,pair 1-oer,Reusable,Expert,norm1,GPT4o,deepseek,mpnet1,pair 1-score,Column0,Column1,Column2,Column3
0,15,72,yes,5,1.00,0.7,0.70,0.7773,0.638177,0.001250,0.001250,0.000689,0.001818
1,15,72,no,2,0.25,0.7,0.70,0.7773,0.638177,0.002812,0.002812,0.003862,0.002093
2,139,147,maybe,4,0.75,0.7,0.65,0.7300,0.509994,0.000035,0.000139,0.000006,0.000800
3,124,113,yes,3,0.50,0.6,0.58,0.6089,0.533602,0.000139,0.000089,0.000165,0.000016
4,41,144,no,2,0.25,0.5,0.55,0.6711,0.460253,0.000868,0.001250,0.002463,0.000614
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,212,209,no,2,0.25,0.4,0.25,0.4160,0.457196,0.000313,0.000000,0.000383,0.000596
68,194,418,no,1,0.00,0.5,0.55,0.5458,0.491961,0.003472,0.004201,0.004137,0.003361
69,52,253,no,1,0.00,0.1,0.20,0.1375,0.030290,0.000139,0.000556,0.000263,0.000013
70,1,428,no,1,0.00,0.2,0.15,0.1418,0.131511,0.000556,0.000313,0.000279,0.000240


In [5]:
ground_truth = survey['norm1'].values
gpt4o_eval = survey['GPT4o'].values
deepseek_eval = survey['deepseek'].values
mpnet_eval = survey['mpnet1'].values
finetuned_eval = survey['pair 1-score'].values

In [8]:
# Pearson and Spearman correlations (existing code)
pearson_mpnet, _ = pearsonr(ground_truth, mpnet_eval)
spearman_mpnet, _ = spearmanr(ground_truth, mpnet_eval)
mse_mpnet = mean_squared_error(ground_truth, mpnet_eval)
r2_mpnet = r2_score(ground_truth, mpnet_eval)

pearson_finetuned, _ = pearsonr(ground_truth, finetuned_eval)
spearman_finetuned, _ = spearmanr(ground_truth, finetuned_eval)
mse_finetuned = mean_squared_error(ground_truth, finetuned_eval)
r2_finetuned = r2_score(ground_truth, finetuned_eval)

pearson_gpt4o, _ = pearsonr(ground_truth, gpt4o_eval)
spearman_gpt4o, _ = spearmanr(ground_truth, gpt4o_eval)
mse_gpt4o = mean_squared_error(ground_truth, gpt4o_eval)
r2_gpt4o = r2_score(ground_truth, gpt4o_eval)

pearson_deepseek, _ = pearsonr(ground_truth, deepseek_eval)
spearman_deepseek, _ = spearmanr(ground_truth, deepseek_eval)
mse_deepseek = mean_squared_error(ground_truth, deepseek_eval)
r2_deepseek = r2_score(ground_truth, deepseek_eval)

print(f"Finetuned - Pearson: {pearson_finetuned}, Spearman: {spearman_finetuned}, MSE: {mse_finetuned}, R2: {r2_finetuned}")
print(f"MPNet - Pearson: {pearson_mpnet}, Spearman: {spearman_mpnet}, MSE: {mse_mpnet}, R2: {r2_mpnet}")
print(f"gpt4o - Pearson: {pearson_gpt4o}, Spearman: {spearman_gpt4o}, MSE: {mse_gpt4o}, R2: {r2_gpt4o}")
print(f"deepseek - Pearson: {pearson_deepseek}, Spearman: {spearman_deepseek}, MSE: {mse_deepseek}, R2: {r2_deepseek}")

Finetuned - Pearson: 0.5976809640768012, Spearman: 0.584383021624903, MSE: 0.06313798397093406, R2: 0.3531476108590471
MPNet - Pearson: 0.5649175917020225, Spearman: 0.5959606110621715, MSE: 0.07514060222222223, R2: 0.23018007525691708
gpt4o - Pearson: 0.5957333989723446, Spearman: 0.6124893285101906, MSE: 0.06541666666666668, R2: 0.32980237154150194
deepseek - Pearson: 0.5377076602913045, Spearman: 0.555013397263884, MSE: 0.07325694444444444, R2: 0.24947826086956537
